In [1]:
!pip install pmdarima

     --------------------------------------- 14.9/14.9 MB 11.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\jason\\anaconda3\\Lib\\site-packages\\~umpy.libs\\libscipy_openblas64_-caad452230ae4ddb57899b8b3a33c55c.dll'
Consider using the `--user` option or check the permissions.



In [3]:
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
pd.set_option('display.float_format', '{:.5f}'.format)

# Data Cleaning

In [23]:
energy_data = pd.read_excel("DATA/sales_revenue.xlsx")
energy_data = energy_data.drop(index = 0)
energy_data = energy_data.reset_index(drop=True)
energy_data.columns = energy_data.iloc[0]
energy_data = energy_data.iloc[1:]
energy_data = energy_data.reset_index(drop=True)
energy_data = energy_data.iloc[:, [0,1,2,20,21,22,23]]
energy_data.drop(index = 9690, inplace = True)

In [24]:
# COLUMN NAMES
print(energy_data.columns.tolist())
energy_data = energy_data.rename(columns = {
    'Thousand Dollars' : 'revenue',
    'Megawatthours' : 'sales',
    'Count' : 'customer_count',
    'Cents/kWh' : 'price'
})
energy_data.columns = energy_data.columns.str.strip().str.lower()

print(energy_data.columns.tolist())

['Year', 'Month', 'State', 'Thousand Dollars', 'Megawatthours', 'Count', 'Cents/kWh']
['year', 'month', 'state', 'revenue', 'sales', 'customer_count', 'price']


In [25]:
# COLUMN TYPES
print(energy_data.info())

vars = ['year', 'month' , 'revenue' , 'sales' , 'customer_count' , 'price']
for col in vars:
    energy_data[col] = pd.to_numeric(energy_data[col], errors='coerce')  # converts, turns non-numeric to NaN
print(energy_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9690 entries, 0 to 9689
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            9690 non-null   object
 1   month           9690 non-null   object
 2   state           9690 non-null   object
 3   revenue         9690 non-null   object
 4   sales           9690 non-null   object
 5   customer_count  9690 non-null   object
 6   price           9690 non-null   object
dtypes: object(7)
memory usage: 530.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9690 entries, 0 to 9689
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            9690 non-null   int64  
 1   month           9690 non-null   int64  
 2   state           9690 non-null   object 
 3   revenue         9690 non-null   float64
 4   sales           9690 non-null   float64
 5   customer_count  9690 non-

In [26]:
# CONVERT TO NATIONAL DATA
energy_data = energy_data.groupby(['year' , 'month'], as_index = False).sum()
#energy_data.drop(columns = ['state'], inplace = True)


In [27]:
print(energy_data.head(5))
energy_data['date'] = pd.to_datetime(energy_data[['year', 'month']].assign(day = 1))
energy_data.set_index('date', inplace = True)
print(energy_data.head(5))

0  year  month        revenue           sales  customer_count     price
0  2010      1 30783698.00000 331878766.00000       143201001 494.02000
1  2010      2 28266233.00000 298459729.00000       143096373 501.05000
2  2010      3 27720476.00000 292511350.00000       144193891 501.29000
3  2010      4 25452822.00000 267052130.00000       144139020 502.22000
4  2010      5 27588157.00000 283729266.00000       143683686 510.83000
0           year  month        revenue           sales  customer_count  \
date                                                                     
2010-01-01  2010      1 30783698.00000 331878766.00000       143201001   
2010-02-01  2010      2 28266233.00000 298459729.00000       143096373   
2010-03-01  2010      3 27720476.00000 292511350.00000       144193891   
2010-04-01  2010      4 25452822.00000 267052130.00000       144139020   
2010-05-01  2010      5 27588157.00000 283729266.00000       143683686   

0              price  
date                  
201

In [28]:
train_data = energy_data[energy_data['year'] <= 2022]
test_data = energy_data[energy_data['year'] > 2022]
train_data = train_data[['sales']]
test_data = test_data[['sales']]

# EDA

In [29]:

import plotly.express as px


train_part = energy_data[energy_data.index.year < 2023]
test_part = energy_data[energy_data.index.year >= 2023]


dates_pred = test_part.index



fig = px.line()


fig.add_scatter(x=train_part.index, y=train_part.sales,
                mode='lines', line=dict(color='black'), name='train_data')


fig.add_scatter(x=test_part.index, y=test_part.sales,
                mode='lines', line=dict(color='blue'), name='test_data')



fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Demand History Data with Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)'
)

fig.show()

C:\Users\jason\anaconda3\lib\site-packages\plotly\io\_renderers.py:51: UserWarning:

Plotly version >= 6 requires Jupyter Notebook >= 7 but you have 6.4.12 installed.
 To upgrade Jupyter Notebook, please run `pip install notebook --upgrade`.



In [ ]:

fig, ax = plt.subplots(figsize=(20,6))  # control size here
plot_acf(train_data['sales'], lags=100, ax=ax)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(20,6))  # control size here
plot_pacf(train_data['sales'], lags=50, ax=ax)
plt.show()

# MODEL FITTING


## AUTO ARIMA BASELINE

In [ ]:
# Auto Arima

from sklearn.preprocessing import StandardScaler
import pmdarima as pm


scaler = StandardScaler()
sales_scaled_array = scaler.fit_transform(train_data[['sales']])  # NumPy array
sales_train_scaled = pd.Series(sales_scaled_array.flatten(), index=train_data.index, name='sales')


model = pm.auto_arima(
    sales_train_scaled,
    seasonal=True,
    m=12,
    stepwise=True,
    suppress_warnings=True
)


print(model.summary())

# SARIMAX(1, 0, 0)x(2, 1, 0, 12)

## FIRST MODEL

In [ ]:
# SARIMAX(1, 0, 0)x(2, 1, 0, 12)

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler


model_manual = SARIMAX(
    sales_train_scaled,
    order=(1,0,0),
    seasonal_order=(2,1,0,12),
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_1 = model_manual.fit(disp=False)

print(model_1.summary())

y_train_pred_scaled = model_1.fittedvalues
y_train_pred = scaler.inverse_transform(y_train_pred_scaled.values.reshape(-1, 1)).flatten()

n_test = len(test_data)
y_test_pred_scaled = model_1.get_forecast(steps=n_test).predicted_mean


y_test_pred = scaler.inverse_transform(y_test_pred_scaled.values.reshape(-1, 1)).flatten()


# MAPE

y_true = test_data['sales'].values
mape = np.mean(np.abs((y_true - y_test_pred) / y_true)) * 100
print(f"Test MAPE: {mape:.2f}%")


In [ ]:
import plotly.express as px
import plotly.graph_objects as go


train_dates = train_data.index
train_sales = train_data['sales']


y_train_pred_1d = y_train_pred  


fig = go.Figure()


fig.add_trace(go.Scatter(
    x=train_dates,
    y=train_sales,
    mode='lines',
    line=dict(color='black'),
    name='train_data'
))


fig.add_trace(go.Scatter(
    x=train_dates,  
    y=y_train_pred_1d,
    mode='lines',
    line=dict(color='red'),
    name='predicted_train'
))


fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Train Data with In-Sample SARIMA Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.show()


In [ ]:
import plotly.express as px




dates_pred = test_part.index



fig = px.line()  

fig.add_scatter(x=train_data.index, y=train_data.sales,
                mode='lines', line=dict(color='black'), name='train_data')


fig.add_scatter(x=test_data.index, y=test_data.sales,
                mode='lines', line=dict(color='blue'), name='test_data')



fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Demand History Data with Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)'
)

fig.add_scatter(
    x=test_data.index,
    y=y_test_pred,
    mode='lines',
    line=dict(color='red'),
    name='test_predictions'
)
fig.show()


## Second Model

In [ ]:


from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler


model_manual = SARIMAX(
    sales_train_scaled,
    order=(1,1,0),
    seasonal_order=(2,1,0,12),
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_2 = model_manual.fit(disp=False)

print(model_2.summary())

y_train_pred_scaled = model_2.fittedvalues
y_train_pred = scaler.inverse_transform(y_train_pred_scaled.values.reshape(-1, 1)).flatten()

n_test = len(test_data)
y_test_pred_scaled = model_2.get_forecast(steps=n_test).predicted_mean


y_test_pred = scaler.inverse_transform(y_test_pred_scaled.values.reshape(-1, 1)).flatten()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


train_dates = train_data.index
train_sales = train_data['sales']


y_train_pred_1d = y_train_pred  

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=train_dates,
    y=train_sales,
    mode='lines',
    line=dict(color='black'),
    name='train_data'
))


fig.add_trace(go.Scatter(
    x=train_dates,
    y=y_train_pred_1d,
    mode='lines',
    line=dict(color='red'),
    name='predicted_train'
))

# Layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Train Data with In-Sample SARIMA Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.show()


In [ ]:
import plotly.express as px

# Split data


# x-axis for predictions = test period
dates_pred = test_part.index

# Make sure y_pred is 1D
#y_pred_1d = y_pred.ravel()

fig = px.line()  # empty figure

# Training portion (black)
fig.add_scatter(x=train_data.index, y=train_data.sales,
                mode='lines', line=dict(color='black'), name='train_data')

# Test portion (blue)
fig.add_scatter(x=test_data.index, y=test_data.sales,
                mode='lines', line=dict(color='blue'), name='test_data')


# Layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Demand History Data with Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.add_scatter(
    x=test_data.index,
    y=y_test_pred,
    mode='lines',
    line=dict(color='red'),
    name='test_predictions'
)
fig.show()

## Third Model

In [ ]:
# SARIMAX(1, 1, 1)x(2, 1, 0, 12)

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler


model_manual = SARIMAX(
    sales_train_scaled,
    order=(1,1,1),
    seasonal_order=(2,1,0,12),
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_3 = model_manual.fit(disp=False)

print(model_3.summary())

y_train_pred_scaled = model_3.fittedvalues
y_train_pred = scaler.inverse_transform(y_train_pred_scaled.values.reshape(-1, 1)).flatten()

n_test = len(test_data)
y_test_pred_scaled = model_3.get_forecast(steps=n_test).predicted_mean


y_test_pred = scaler.inverse_transform(y_test_pred_scaled.values.reshape(-1, 1)).flatten()

In [ ]:
residuals = model_3.resid

# ACF plot
plot_acf(residuals, lags=40)
plt.title("ACF of SARIMA Residuals")
plt.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Dates and original sales
train_dates = train_data.index
train_sales = train_data['sales']


y_train_pred_1d = y_train_pred  # 1D array matching train_dates

# Create figure
fig = go.Figure()

# Actual train data in black
fig.add_trace(go.Scatter(
    x=train_dates,
    y=train_sales,
    mode='lines',
    line=dict(color='black'),
    name='train_data'
))

# Predicted train data in red
fig.add_trace(go.Scatter(
    x=train_dates,  # full-length predictions align with train_dates
    y=y_train_pred_1d,
    mode='lines',
    line=dict(color='red'),
    name='predicted_train'
))

# Layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Train Data with In-Sample SARIMA Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.show()

In [ ]:
import plotly.express as px

# Split data


# x-axis for predictions = test period
dates_pred = test_part.index

# Make sure y_pred is 1D
#y_pred_1d = y_pred.ravel()

fig = px.line()  # empty figure

# Training portion (black)
fig.add_scatter(x=train_data.index, y=train_data.sales,
                mode='lines', line=dict(color='black'), name='Training Data')

# Test portion (blue)
fig.add_scatter(x=test_data.index, y=test_data.sales,
                mode='lines', line=dict(color='blue'), name='Test Data')


# Layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Demand History Data with Predictions",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.add_scatter(
    x=test_data.index,
    y=y_test_pred,
    mode='lines',
    line=dict(color='red'),
    name='Predicted Test Data'
)
fig.show()

# Forecasting

## Model 1

In [ ]:
n_future = 24
future_pred_scaled = model_1.get_forecast(steps=n_future).predicted_mean


future_pred = scaler.inverse_transform(future_pred_scaled.values.reshape(-1, 1)).flatten()

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Full dataset dates and sales
full_dates = energy_data.index
full_sales = energy_data['sales']

# Future dates for 24-month forecast
last_date = energy_data.index[-1]
future_dates = pd.date_range(
    start=last_date + pd.offsets.MonthBegin(1),
    periods=n_future,
    freq='MS'
)

# Create figure
fig = go.Figure()

# Original full dataset in black
fig.add_trace(go.Scatter(
    x=full_dates,
    y=full_sales,
    mode='lines',
    line=dict(color='black'),
    name='historical_sales'
))

# Forecast in green
fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_pred,
    mode='lines',
    line=dict(color='green'),
    name='forecast_2_years'
))

# Layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Data with 2-Year SARIMA Forecast",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.show()

## Model 2

In [ ]:
n_future = 24
future_pred_scaled = model_2.get_forecast(steps=n_future).predicted_mean


future_pred = scaler.inverse_transform(future_pred_scaled.values.reshape(-1, 1)).flatten()



In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Full dataset dates and sales
full_dates = energy_data.index
full_sales = energy_data['sales']

# Future dates for 24-month forecast
last_date = energy_data.index[-1]
future_dates = pd.date_range(
    start=last_date + pd.offsets.MonthBegin(1),
    periods=n_future,
    freq='MS'
)

# Create figure
fig = go.Figure()

# Original full dataset in black
fig.add_trace(go.Scatter(
    x=full_dates,
    y=full_sales,
    mode='lines',
    line=dict(color='black'),
    name='historical_sales'
))

# Forecast in green
fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_pred,
    mode='lines',
    line=dict(color='green'),
    name='forecast_2_years'
))

# Layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Data with 2-Year SARIMA Forecast",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)',
    height = 700
)

fig.show()

## Model 3

In [ ]:

n_future = 24
future_pred_scaled = model_3.get_forecast(steps=n_future).predicted_mean


future_pred = scaler.inverse_transform(future_pred_scaled.values.reshape(-1, 1)).flatten()


In [ ]:
import plotly.graph_objects as go
import pandas as pd


full_dates = energy_data.index
full_sales = energy_data['sales']


last_date = energy_data.index[-1]
future_dates = pd.date_range(
    start=last_date + pd.offsets.MonthBegin(1),
    periods=n_future,
    freq='MS'
)


fig = go.Figure()


fig.add_trace(go.Scatter(
    x=full_dates,
    y=full_sales,
    mode='lines',
    line=dict(color='black'),
    name='historical_sales'
))


fig.add_trace(go.Scatter(
    x=future_dates,
    y=future_pred,
    mode='lines',
    line=dict(color='green'),
    name='forecast_2_years'
))

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Sales",
    title={'text': "Energy Data with 2-Year SARIMA Forecast",
           'y':0.95, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
    plot_bgcolor='rgba(220,220,220,0.8)'
)

fig.show()